**DEVELOPMENT OF A FUNCTION FOR SEARCHING INFO ON STACK OVERFLOW**

Author: Pablo Legerén Somolinos

Definition of both the packages and the necessary libraries.

In [ ]:
!pip install selenium
!pip install --upgrade webdriver_manager

In [18]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

import re
import pandas as pd
import numpy as np
import time
from time import strftime, gmtime

from selenium.common.exceptions import (
    ElementNotSelectableException, 
    ElementNotVisibleException,
    NoSuchElementException,
    TimeoutException, 
    WebDriverException, 
    WebDriverException
    )

import random
import requests

negrita = "\033[1m"
reset = "\033[0m"

A first function is defined in which Google is opened, this is done at first thinking that entering from Google would not ask for the Captcha. 
It is verified that this does not work but it is left like this as the result is the same.

In [20]:
def abrir_google():
    
    search_url = "https://www.google.com/search?q=google&oq=google&gs_lcrp=EgZjaHJvbWUyBggAEEUYOdIBBzkxOWowajGoAgCwAgA&sourceid=chrome&ie=UTF-8"
    
    webd.get(search_url)
    
    try:
        boton_aceptar_todo=webd.find_element(By.CSS_SELECTOR,'button[id="L2AGLb"]')
        boton_aceptar_todo.click()
        
    except NoSuchElementException:
        
        pass

The second function to be defined is the one in charge of opening Stack Overflow, which enters the web from Google. The first thing it does is to delete the word "google" from the search engine to search for "Stack Overflow" and click on the first link (the one that corresponds to the official website).

In [21]:
def abrir_stack():
    
    busqueda_google = webd.find_element(By.CSS_SELECTOR, 'textarea[role="combobox"]')
    
    for _ in range(6):
        busqueda_google.send_keys(Keys.BACK_SPACE)
        time.sleep(random.uniform(0,0.7))
    
    busqueda_google.send_keys('Stack Overflow')
    busqueda_google.send_keys(Keys.ENTER)
    
    #Abrir el primer link
    primer_link = webd.find_element(By.CSS_SELECTOR,'a[href="https://stackoverflow.com/"]')
    primer_link.click()

Once inside, a function is defined that accepts cookies from the website.

In [22]:
def aceptar_cookies():
    
    try:
        
        WebDriverWait(webd,10).until(ec.visibility_of_element_located((By.CSS_SELECTOR,'button[id="onetrust-accept-btn-handler"]')))
        boton_aceptar = webd.find_element(By.CSS_SELECTOR,'button[id="onetrust-accept-btn-handler"]')
        
        boton_aceptar.click()
        
    except (NoSuchElementException, TimeoutException) as ex:
        
        print("No se han podido aceptar las cookies: ",type(ex))
        webd.quit()
        

This "print_final" function will allow a clearer display of what is being requested. In addition to showing the best rated answer, it will also show the question it answers and let the user choose how many answers he/she would like to receive. In case of a number greater than 1, the order will remain as above, from a better rating to a worse one.

In [23]:
def print_final(textos, num_respuestas):
    
    pregunta=BeautifulSoup(str(textos[0].text),'html.parser')
    print(f"\n{negrita}La pregunta realizada fue:{reset}\n{pregunta}\n")
    
    for num in range(min(int(num_respuestas),len(textos))):
                     
        respuesta=BeautifulSoup(str(textos[int(num+1)].text),'html.parser')
        print(f"{negrita}La {num+1} respuesta mejor valorada fue:{reset}\n{str(respuesta)}\n")

Finally, the function get_SOS_help is defined, in which first click on the search bar to enter the value of the argument of the function, this error will give the user the option to enter it on the screen. 

If it is the first time that the user accesses the website, a CAPTCHA will appear. What has been achieved, to minimise human action is that the user only has to select the photos and click on "Verify", to know when to act, the message "RESUELVE EL CAPTCHA MANUALMENTE" will be displayed on the screen.

Once inside, to sort the answers by rating, click on the "more" button and select score, now the answers are sorted from highest to lowest rating.

Finally, both the question and the answers are stored, in case the user wants to obtain more than one, and the desired number is displayed on the screen.

In [24]:
def get_SOS_help(error):
    
    time.sleep(random.uniform(0,3))
    
    boton_buscar=webd.find_element(By.CSS_SELECTOR,'input[role="combobox"]')
    boton_buscar.send_keys(str(error))
    boton_buscar.send_keys(Keys.ENTER)
    
    time.sleep(random.uniform(0,3))
    
    try:
        
        iframe = webd.find_element(By.CSS_SELECTOR, 'iframe[title="reCAPTCHA"]')
        webd.switch_to.frame(iframe)
        boton_capcha=webd.find_element(By.XPATH,'//*[@id="recaptcha-anchor"]/div[1]').click()
        webd.switch_to.default_content()
        print(f"\n{negrita}RESUELVA EL CAPTCHA MANUALMENTE{reset}\n")
        
    except NoSuchElementException:
        
        pass
    
    
    WebDriverWait(webd,10000).until(ec.visibility_of_element_located((By.XPATH,'//*[@id="mainbar"]/div[4]/div[2]/div/nav/button')))
    boton_more=webd.find_element(By.XPATH,'//*[@id="mainbar"]/div[4]/div[2]/div/nav/button')
    boton_more.click()
    
    time.sleep(1)

    boton_score=webd.find_element(By.XPATH,'//*[@id="mainbar"]/div[4]/div[2]/div/nav/ul/li[2]/a')
    boton_score.click()
    
    time.sleep(random.uniform(0,3))
    
    solution = webd.find_elements(By.CSS_SELECTOR, "a[data-searchsession]")
    solution[0].click()

    time.sleep(random.uniform(1.5,2.5))
    
    num_respuestas=input("¿Cuántas respuestas quieres?: ")
    
    textos=webd.find_elements(By.CSS_SELECTOR, 'div[class="s-prose js-post-body"]')
    
    print_final(textos,num_respuestas)
    

**Additional notes**:

-All the times when using the function time.sleep it does not declare a fixed timeout but a random time between two values, this was thought so that the execution time between commands is not the same and it becomes more automatic but more like a "human behaviour" thinking that this way you could skip the CAPTCHA, which has not worked. 

-At the beginning of the project I got a strange error with Chrome so I had to download a specific version which is the one indicated in the "executable_path".

In [ ]:
service = Service(executable_path=r'C:\Users\pablo\Desktop\chromedriver-win32\chromedriver-win32\chromedriver.exe')
options = webdriver.ChromeOptions()
webd = webdriver.Chrome(service=service, options=options)

abrir_google()
abrir_stack()
aceptar_cookies()

error=input("Introduce el error que quieras buscar en Stack Overflow: ")
#error que busco: Can't install time module
get_SOS_help(error)